In [1]:
#IMPORTS

import pandas as pd

In [2]:
#clean main side door spreadsheet
%run create_side_door_by_person_exhibit_binary_df.ipynb
all_doors_df_clean = create_side_door_by_person_exhibit_binary_df.clean_all_door_data(all_doors_df.copy())
all_doors_df_clean.head()

ModuleNotFoundError: No module named 'nbformat'

In [78]:
side_doors_df = create_door_type_dfs(all_doors_df_clean.copy(), "SIDE")
door_close_df = create_door_type_dfs(all_doors_df_clean.copy(), "DC")
messy_middle_df = create_door_type_dfs(all_doors_df_clean.copy(), "MM")

In [79]:
#create binary dummy columns for door occurances in each area of the exhibit
side_doors_df = pd.get_dummies(side_doors_df, columns=['Exhibit #'], drop_first=False, dtype=int)
door_close_df = pd.get_dummies(door_close_df, columns=['Exhibit #'], drop_first=False, dtype=int)
messy_middle_df = pd.get_dummies(messy_middle_df, columns=['Exhibit #'], drop_first=False, dtype=int)

In [80]:
# Written by Claude and Hailey

# Function to create aggregation dictionary for a dataframe
def create_agg_dict(df):
    # Get list of numeric columns that start with "Exhibit #"
    numeric_cols = [col for col in df.columns if col.startswith('Exhibit #')]
    # Get list of non-numeric columns (excluding 'Pseudonym' which is our groupby column)
    other_cols = [col for col in df.columns if col not in numeric_cols and col != 'Pseudonym']
    
    # Create aggregation dictionary
    agg_dict = {col: 'sum' for col in numeric_cols}
    agg_dict.update({col: 'first' for col in other_cols})
    
    return agg_dict

# Check for inconsistent values in non-numeric columns
def check_consistency(df):
    non_numeric_cols = df.select_dtypes(exclude=['number']).columns
    non_numeric_cols = [col for col in non_numeric_cols if col != 'Pseudonym']
    
    for column in non_numeric_cols:
        inconsistent = df.groupby('Pseudonym')[column].nunique().max() > 1
        if inconsistent:
            print(f"Warning: Column {column} has inconsistent values for some participants")
            # Optionally, show the problematic values:
            problematic = df.groupby('Pseudonym')[column].nunique()
            problematic = problematic[problematic > 1]
            print("Participants with inconsistent values:")
            for participant in problematic.index:
                values = df[df['Pseudonym'] == participant][column].unique()
                print(f"{participant}: {values}")

# Apply checks and aggregation to each dataframe
for df_name, df in [('side_doors_df', side_doors_df), 
                   ('door_close_df', door_close_df), 
                   ('messy_middle_df', messy_middle_df)]:
    print(f"\nChecking {df_name}:")
    check_consistency(df)
    
    # Perform the aggregation
    agg_dict = create_agg_dict(df)
    if df_name == 'side_doors_df':
        side_doors_df = df.groupby('Pseudonym').agg(agg_dict).reset_index()
    elif df_name == 'door_close_df':
        door_close_df = df.groupby('Pseudonym').agg(agg_dict).reset_index()
    else:
        messy_middle_df = df.groupby('Pseudonym').agg(agg_dict).reset_index()


Checking side_doors_df:

Checking door_close_df:

Checking messy_middle_df:


In [81]:
display(door_close_df)

,Pseudonym,Exhibit #_1,Exhibit #_2,Exhibit #_3,Exhibit #_4,Exhibit #_5,Exhibit #_6,Exhibit #_7,Exhibit #_8,Exhibit #_9,...,Exhibit #_s4,Group ID,Group Type,age,gender,race,zip,SASSY,UT?,rural/urban
0,Allie,0,0,0,0,0,0,0,0,0,...,0,240313_04,Mixed,28,Female,Caucasian,84401,Concerned,1,2.0
1,Amy,0,0,1,1,0,0,0,0,0,...,0,24040302,CC,20,Female,Caucasian,84095,Cautious,1,1.0
2,Andy,0,0,1,0,0,0,0,0,0,...,0,24012401,mixed,23,Male,white,30338,Cautious,0,1.0
3,Barney,0,0,0,0,0,0,0,1,0,...,0,24040607,mixed,33,Male,caucasian,49504,Concerned,0,1.0
4,Bill,0,2,0,1,0,0,0,0,0,...,0,24012601,Mixed,59,Male,Caucasian,84074,Doubtful,1,1.0
5,Billy,0,0,0,2,0,0,0,1,0,...,0,240313_02,Mixed,32,Male,Caucasian,Winnipeg,Concerned,0,NaN
6,Brenda,0,2,0,1,0,0,0,0,0,...,0,24040607,mixed,34,Female,caucasian,84115,Dismissive,1,1.0
7,Caleb,0,1,0,0,0,0,0,0,0,...,0,24.02.23_02,Mixed Group,19,Male,Caucasian,R3K0B4,Doubtful,0,1.0
8,Cheryl,0,0,0,0,0,0,0,0,1,...,0,24022803,mixed,47,Female,White,83642,Cautious,0,2.0
9,Daniel,0,1,0,0,0,0,0,0,0,...,0,240313_04,Mixed,27,Male,Caucasian,84401,Cautious,1,2.0


In [82]:
display(side_doors_df)

,Pseudonym,Exhibit #_1,Exhibit #_2,Exhibit #_3,Exhibit #_4,Exhibit #_5,Exhibit #_6,Exhibit #_7,Exhibit #_8,Exhibit #_9,...,Exhibit #_s4,Group ID,Group Type,age,gender,race,zip,SASSY,UT?,rural/urban
0,Alex,0,0,0,1,2,0,0,0,0,...,0,24040505,AC,35,Male,Black or African American,47904,Alarmed,0,3.0
1,Allie,0,4,0,1,1,0,0,0,0,...,0,240313_04,Mixed,28,Female,Caucasian,84401,Concerned,1,2.0
2,Amy,0,0,0,1,4,2,0,0,0,...,0,24040302,CC,20,Female,Caucasian,84095,Cautious,1,1.0
3,Andy,0,1,1,4,0,1,0,0,0,...,0,24012401,mixed,23,Male,white,30338,Cautious,0,1.0
4,Asher,0,0,0,1,0,0,0,0,0,...,0,24.02.23_02,Mixed Group,17,Male,Caucasian,R3K0B4,Cautious,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,Tam,0,0,0,0,0,0,0,0,0,...,0,240313_02,Mixed,28,Male,Asian,Winnipeg,Cautious,0,NaN
68,Ted,0,0,0,2,0,0,2,0,0,...,0,24040608,AC,29,Male,"Other, please specify",84119,Alarmed,1,1.0
69,Terry,0,0,0,0,0,0,0,0,0,...,0,24030804,AC,20,Male,Caucasian,59725,Concerned,0,9.0
70,Tim,0,1,0,0,0,0,0,0,0,...,0,24020903,AC,37,Male,Asian,98264,Alarmed,0,3.0


In [83]:
display(messy_middle_df)

,Pseudonym,Exhibit #_2,Exhibit #_3,Exhibit #_4,Exhibit #_5,Exhibit #_6,Exhibit #_7,Exhibit #_9,Exhibit #_10,Exhibit #_11,...,Exhibit #_s3,Group ID,Group Type,age,gender,race,zip,SASSY,UT?,rural/urban
0,Allie,0,0,0,1,0,0,0,0,1,...,0,240313_04,Mixed,28,Female,Caucasian,84401,Concerned,1,2.0
1,Andy,0,0,0,0,0,0,0,1,0,...,0,24012401,mixed,23,Male,white,30338,Cautious,0,1.0
2,Billy,0,0,0,0,0,0,0,1,0,...,0,240313_02,Mixed,32,Male,Caucasian,Winnipeg,Concerned,0,NaN
3,Daniel,0,0,0,1,0,0,0,0,0,...,0,240313_04,Mixed,27,Male,Caucasian,84401,Cautious,1,2.0
4,Eddie,1,0,0,0,0,0,0,0,0,...,0,24040505,AC,29,Male,Caucasian,47904,Alarmed,0,3.0
5,Emily,0,0,1,0,0,0,0,0,0,...,0,24.02.02_03,Mixed,46,Female,White,84070,Alarmed,1,1.0
6,Ethan,0,0,0,0,0,0,0,0,0,...,0,24.01.26_02,AC,30,Male,white,84104,Alarmed,1,1.0
7,Frank,1,0,0,0,0,0,0,1,0,...,0,24.02.09_02,AC,33,Male,Caucasian,84115,Alarmed,1,1.0
8,Gina,0,0,0,0,0,0,0,1,0,...,0,24030804,AC,19,Female,Caucasian,59725,Concerned,0,9.0
9,Henry,0,0,0,0,0,0,0,1,0,...,0,24.02.23_02,Mixed Group,54,Male,Caucasian,R3K0B4,Alarmed,0,1.0


In [84]:
#rename exhibit columns so that they will have correct suffix when merged
messy_middle_df = messy_middle_df.rename(columns={col: f"{col}_mm" for col in messy_middle_df.columns if 'Exhibit #' in col})
side_doors_df = side_doors_df.rename(columns={col: f"{col}_side" for col in side_doors_df.columns if 'Exhibit #' in col})
door_close_df = door_close_df.rename(columns={col: f"{col}_dc" for col in door_close_df.columns if 'Exhibit #' in col})

total_doors_by_person = (pd.merge(side_doors_df, 
                                    door_close_df, 
                                    on='Pseudonym', 
                                    how='outer', suffixes=('', '_x'))
                                .merge(messy_middle_df,
                                    on='Pseudonym',
                                    how='outer', suffixes=('', '_y')))

In [85]:
total_doors_by_person = remove_duplicates(total_doors_by_person, '_y')
total_doors_by_person = remove_duplicates(total_doors_by_person, '_x')

#fill null values with 0
total_doors_by_person = total_doors_by_person.fillna(0)

In [86]:
#remove extra characters in age
total_doors_by_person['age'] = total_doors_by_person['age'].str.extract(r'(\d+)')

# Convert number columns to int
columns_to_convert = ['UT?', 'rural/urban', 'age'] + [col for col in total_doors_by_person.columns if "Exhibit #" in col]
total_doors_by_person[columns_to_convert] = total_doors_by_person[columns_to_convert].astype(int)

#add missing columns and fill with zeros
total_doors_by_person[['Exhibit #_1_mm','Exhibit #_8_mm', 'Exhibit #_14_mm', 'Exhibit #_s1_mm', 'Exhibit #_s2_mm', 'Exhibit #_s4_mm']] = 0

In [87]:
# Get lists of columns for each type of door
side_cols = [col for col in total_doors_by_person.columns if col.endswith('_side')]
dc_cols = [col for col in total_doors_by_person.columns if col.endswith('_dc')]
mm_cols = [col for col in total_doors_by_person.columns if col.endswith('_mm')]

#create columns for total doors of each type
total_doors_by_person['total_side'] = total_doors_by_person[side_cols].sum(axis=1)
total_doors_by_person['total_dc'] = total_doors_by_person[dc_cols].sum(axis=1)
total_doors_by_person['total_mm'] = total_doors_by_person[mm_cols].sum(axis=1)

#create columns for total of all doors
total_doors_by_person['total_all_doors'] = total_doors_by_person[['total_side', 'total_dc', 'total_mm']].sum(axis=1)


In [88]:
display(total_doors_by_person)

,Pseudonym,Exhibit #_1_side,Exhibit #_2_side,Exhibit #_3_side,Exhibit #_4_side,Exhibit #_5_side,Exhibit #_6_side,Exhibit #_7_side,Exhibit #_8_side,Exhibit #_9_side,...,Exhibit #_1_mm,Exhibit #_8_mm,Exhibit #_14_mm,Exhibit #_s1_mm,Exhibit #_s2_mm,Exhibit #_s4_mm,total_side,total_dc,total_mm,total_all_doors
0,Alex,0,0,0,1,2,0,0,0,0,...,0,0,0,0,0,0,8,0,0,8
1,Allie,0,4,0,1,1,0,0,0,0,...,0,0,0,0,0,0,11,2,2,15
2,Amy,0,0,0,1,4,2,0,0,0,...,0,0,0,0,0,0,12,4,0,16
3,Andy,0,1,1,4,0,1,0,0,0,...,0,0,0,0,0,0,9,7,1,17
4,Asher,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,Tam,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
72,Ted,0,0,0,2,0,0,2,0,0,...,0,0,0,0,0,0,7,0,1,8
73,Terry,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,2,0,4
74,Tim,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,3,0,6


In [89]:
#save total doors by person to excel file
total_doors_by_person.to_excel('c:/Users/Hailey Sherman/ClimateOfHopeData/Data_2025/total_doors_by_person.xlsx', index=False)